In [1]:
 conda install -c anaconda lxml 


Solving environment: done

## Package Plan ##

  environment location: /Users/devtechbox/opt/anaconda3

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.3.29  |       hecd8cb5_1         125 KB  anaconda
    certifi-2021.10.8          |   py39hecd8cb5_2         156 KB  anaconda
    openssl-1.1.1n             |       hca72f7f_0         3.5 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.8 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                                 pkgs/main --> anaconda None
  certifi                                         pkgs/main --> anaconda None
  openssl                                         pkgs/main --> anaconda None



certifi-2021.10.8    | 156 KB    | #############

In [2]:
conda install -c anaconda beautifulsoup4 


Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
 conda install -c anaconda requests 


Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [13]:
from bs4 import BeautifulSoup as bs
import requests
import csv

In [15]:
#first call API
url="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales"
response = requests.get(url)
html = response.content
# ameliorer l'affichage avec BS
soup = bs(html, "lxml")

In [16]:
#Recuperer le nombre de pages à parcourir pour avoir tous les URL des articles
pages_number= int(soup.find("li", class_="pagination__text").get_text().replace("\n", "").split(' ')[7])
print(pages_number)


2


In [19]:
i=1
urlList=[]
#recuperer tous les Url des articles et les stocker dans un tableau
while i <= pages_number:
    urlPage="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales?page="+str(i)
    response = requests.get(urlPage)
    html = response.content
    soup = bs(html, "lxml")
    urls=soup.find_all("a", class_="grid-view-item__link")
    for url in urls:
        urlList.append('https://satoriz-comboire.bio'+url['href'])
    i+= 1

In [20]:

#fonction qui parcours chaque url de la liste pour recuperer la data démandée, la nettoyer et la stocker dans des listes
def product_obj(url):
    dataObj = [];
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    dataObj.append(url) ;
    dataObj.append(soup.find("h1", class_="product-single__title").get_text())
    dataObj.append(soup.find("div", class_="product__vendor").get_text().split("\n")[1].split("  ")[6])
    dataObj.append(soup.find("span", class_="rqp").get_text().split("/")[0])
    dataObj.append(soup.find("img", class_="feature-row__image")["src"] if soup.find("img", class_="feature-row__image") else "")
    dataObj.append(soup.find("div", class_="product__refs").get_text().split(": ")[2].split("\n")[0])
    return dataObj

#définir une lite de headers pour le fichier csv
headers=["product Url","product Title", "product Brand","product Price", "product Image Url","product Ref"]
scrapedData=[]
#appeler la fonction avec chaue url de la liste
for product_url in urlList:
    scrapedData.append(product_obj(product_url))
    


In [21]:
print(len(scrapedData))
#exporter les résultats dans un fichier CSV
with open('scrappedData.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(headers)
    write.writerows(scrapedData)


63
